In [1]:
!pip install langchain openai pypdf faiss-cpu tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import io
from urllib.request import Request, urlopen

from pypdf import PdfReader, PdfFileWriter, PdfFileReader

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import (
    ElasticVectorSearch, Pinecone, Weaviate, FAISS
)
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

## OpenAI API

In [3]:
os.environ["OPENAI_API_KEY"] = "API_KEY"

## Program

### Load PDF File as Text String

In [4]:
pdf_url = 'https://s3.amazonaws.com/static.nomic.ai/gpt4all/2023_GPT4All_Technical_Report.pdf'
remote_file = urlopen(Request(pdf_url)).read()
memory_file = io.BytesIO(remote_file)
reader = PdfReader(memory_file)
reader

In [5]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [6]:
raw_text[:100]

'GPT4All: Training an Assistant-style Chatbot with Large Scale Data\nDistillation from GPT-3.5-Turbo\nY'

### Text Splitting

In [7]:
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)

text_ls = text_splitter.split_text(raw_text)

In [8]:
len(text_ls)

8

In [9]:
text_ls[0]

'GPT4All: Training an Assistant-style Chatbot with Large Scale Data\nDistillation from GPT-3.5-Turbo\nYuvanesh Anand\nyuvanesh@nomic.aiZach Nussbaum\nzanussbaum@gmail.com\nBrandon Duderstadt\nbrandon@nomic.aiBenjamin Schmidt\nben@nomic.aiAndriy Mulyar\nandriy@nomic.ai\nAbstract\nThis preliminary technical report describes the\ndevelopment of GPT4All, a chatbot trained\nover a massive curated corpus of assistant in-\nteractions including word problems, story de-\nscriptions, multi-turn dialogue, and code. We\nopenly release the collected data, data cura-\ntion procedure, training code, and final model\nweights to promote open research and repro-\nducibility. Additionally, we release quantized\n4-bit versions of the model allowing virtually\nanyone to run the model on CPU.\n1 Data Collection and Curation\nWe collected roughly one million prompt-\nresponse pairs using the GPT-3.5-Turbo OpenAI\nAPI between March 20, 2023 and March 26th,\n2023. To do this, we first gathered a diverse sam-'

In [10]:
text_ls[1]

'We collected roughly one million prompt-\nresponse pairs using the GPT-3.5-Turbo OpenAI\nAPI between March 20, 2023 and March 26th,\n2023. To do this, we first gathered a diverse sam-\nple of questions/prompts by leveraging three pub-\nlicly available datasets:\n• The unified chip2 subset of LAION OIG.\n• Coding questions with a random sub-sample\nof Stackoverflow Questions\n• Instruction-tuning with a sub-sample of Big-\nscience/P3\nWe chose to dedicate substantial attention to data\npreparation and curation based on commentary in\nthe Stanford Alpaca project (Taori et al., 2023).\nUpon collection of the initial dataset of prompt-\ngeneration pairs, we loaded data into Atlas for data\ncuration and cleaning. With Atlas, we removed all\nexamples where GPT-3.5-Turbo failed to respond\nto prompts and produced malformed output. This\nreduced our total number of examples to 806,199\nhigh-quality prompt-generation pairs. Next, we\ndecided to remove the entire Bigscience/P3 sub-'

### Embeddings

In [11]:
embeddings = OpenAIEmbeddings()
embeddings

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-embedding-ada-002', document_model_name='text-embedding-ada-002', query_model_name='text-embedding-ada-002', embedding_ctx_length=8191, openai_api_key=None, openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=6)

### Vector Store

In [12]:
docsearch = FAISS.from_texts(text_ls, embeddings)
docsearch

### Large Language Model

In [13]:
llm = OpenAI(model_name='text-davinci-003') # text-davinci-003
llm

OpenAI(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f021d6a7ca0>, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.7, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key=None, openai_organization=None, batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False)

### Create Your Chain

In [14]:
chain = load_qa_chain(llm, chain_type="stuff")
chain

StuffDocumentsChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f021d6a7ca0>, verbose=False, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f021d6a7ca0>, verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:", template_format='f-string', validate_template=True), llm=OpenAI(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f021d6a7ca0>, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.7, max_tokens=256, top_p=1, frequency

### Question and Answer

In [15]:
query = "Who are the authors of the article?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Hugo Touvron, Thibaut Lavril, Gautier Izacard, Xavier Martinet, Marie-Anne Lachaux, Timoth ´ee Lacroix, Baptiste Rozi `ere, Naman Goyal, Eric Hambro, Faisal Azhar, Aurelien Rodriguez, Armand Joulin, Edouard Grave, and Guillaume Lample.'

In [16]:
query = "What was the cost of training the GPT4All model?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' $800 in GPU costs (rented from Lambda Labs and Paperspace) including several failed trains, and $500 in OpenAI API spend.'

In [17]:
query = "How was the model trained?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The model was trained with LoRA (Hu et al., 2021) on the 437,605 post-processed examples for four epochs. Detailed model hyper-parameters and training code can be found in the associated repository and model training log.'

In [18]:
query = "What was the size of the training dataset?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The size of the final training dataset was 437,605 prompt-generation pairs.'

In [19]:
query = "How is this different from other models?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" GPT4All is based on LLaMA and has a non-commercial license. It is trained with LoRA and has data from OpenAI's GPT-3.5-Turbo. It is intended only for research purposes and any commercial use is prohibited."

In [20]:
query = "What is Google Bard?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" I don't know."

## Dependencies

In [21]:
!pip install session-info

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import session_info

session_info.show()